# Imports

In [ ]:
import numpy as np
from scipy.io import loadmat, whosmat, savemat
import os
import re
from tqdm import tqdm
import pickle
import warnings
warnings.filterwarnings('ignore')

# Utils

In [ ]:
base_path = ''

In [ ]:
# extract
def ends_with_specific_number(input_string, specific_number):
  pattern = re.compile(rf".*{specific_number}$")
  return bool(pattern.match(input_string))

In [ ]:
'''
Sorts files based on:
  1. subject (Asc)
  2. experiment date (Asc)
'''

def custom_sort(file_name):
  # Extract the number before the underscore and the date
  match = re.match(r'(\d+)_(\d+)', file_name)
  if match:
    number_part = int(match.group(1))
    date_part = int(match.group(2))
    return (number_part, date_part)
  else:
    return (float('inf'), float('inf'))

In [ ]:
features_info = whosmat(os.path.join(base_path, 'ExtractedFeatures/1_20131027.mat'))
unique_features = set([re.sub(r'\d+$', '', i[0]) for i in features_info])
features_info = dict([(i[0], i[1]) for i in features_info])

In [ ]:
# name of features
features_map = {
  0: 'asm_LDS',
  1: 'asm_movingAve',
  2: 'dasm_LDS',
  3: 'dasm_movingAve',
  4: 'dcau_LDS',
  5: 'dcau_movingAve',
  6: 'de_LDS',
  7: 'de_movingAve',
  8: 'psd_LDS',
  9: 'psd_movingAve',
  10: 'rasm_LDS',
  11: 'rasm_movingAve'
}

# Saving All Features Data to Pickle File

In [ ]:
# (15 subj x 3 exp x 15 tri, 12 uniq features, 62 max num channel, 265 max lenght, 5 freq band )
data = np.zeros((675,12,62,265,5))
p = os.path.join(base_path, 'ExtractedFeatures')
for j, file in tqdm(enumerate(sorted([f for f in os.listdir(p) if '.mat' in f and '_' in f], key=custom_sort))): # subj
  # print(j,file)
  mat = loadmat(os.path.join(p, file))
  for i in range(0,15): #trail
    for k in range(12): # feature
      data[j*15+i,k,:features_info[f'{features_map[k]}{i+1}'][0],:features_info[f'{features_map[k]}{i+1}'][1],:] = mat[f'{features_map[k]}{i+1}']
      '''
      data [current trial,feature, from begiging: number of channels of a feature, from begiging:lenght of data for a feature, all freq bands ] = mat [ a feature of a trial]

      *For a specific freq band:
              data[j*15+i,k,:features_info[f'{features_map[k]}{i+1}'][0],:features_info[f'{features_map[k]}{i+1}'][1],:] = mat[f'{features_map[k]}{i+1}'][:,:,freq band number]


      '''

In [ ]:
with open(os.path.join(base_path, 'alldata.pkl'), 'wb') as file:
    pickle.dump(data, file)